In [5]:
import cv2
import numpy as np
import os

In [6]:
import numpy as np

def distance(x1, x2):
    return np.sqrt(sum((x1 - x2) ** 2))

def knn(train, test, k=5):
    dist = []
    
    for i in range(train.shape[0]):
        #get vector and label
        ix = train[i, :-1]
        iy = train[i, -1]
        #compute distance from test point
        d = distance(test,ix)
        dist.append([d,iy])
        

    # Sort vals based on the distance (first element of the tuple)
    dk = sorted(dist, key=lambda x: x[0])[:k]  # Get the nearest k points
    
    # Extract only the labels of the nearest neighbors
    labels = np.array(dk)[:,-1]

    # Use Counter to count the frequency of each label
    output = np.unique(labels,return_counts = True)
    index = np.argmax(output[1])  
    
    return output[0][index]


In [7]:
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
dataset_path = r"C:\Users\rohit\Jupyter-cn\ml\real time face recognition with knn\selfies_training_data" 
skip = 0
face_data = []
labels = []

class_id = 0 #label for given file
names = {} #mapping btw id - name

#data preparation

for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        
        #create mapping btw class_id and name
        names[class_id] = fx[1:-4]
        data_item = np.load(dataset_path+"\\"+fx)
        face_data.append(data_item)
        
        #create labels for the class
        target = class_id*np.ones((data_item.shape[0],))
        class_id +=1
        labels.append(target)

face_dataset = np.concatenate(face_data,axis=0)
face_labels = np.concatenate(labels,axis=0).reshape((-1,1))

trainset = np.concatenate((face_dataset,face_labels),axis=1)
print(trainset.shape)


    


(12, 30001)


In [8]:
while True:
    ret, frame = cap.read()
    if not ret:
        continue

    faces = face_cascade.detectMultiScale(frame, 1.3, 5)

    for (x, y, w, h) in faces:
        # Get the face region of interest
        offset = 10
        face_section = frame[y-offset:y+h+offset, x-offset:x+w+offset]
        face_section = cv2.resize(face_section, (100, 100))

        out = knn(trainset, face_section.flatten())

        # Display the name and rectangle around the face
        pred_name = names[int(out)]
        cv2.putText(frame, pred_name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 255), 2)

    cv2.imshow("Faces", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
